# 要約 
このJupyter Notebookは、Kaggleの「20の質問」コンペティションに参加するためのHugging Face 8Bまたはそれ以下のサイズの言語モデル（LLM）用のスタートコードを提供しています。主に、質問者と回答者のエージェントを設計・実装することが目的とされています。

### 問題の概要
このノートブックでは、特定のキーワードをバイナリ検索方式で推測するためのエージェントを作成します。質問者の役割は、事前に設定された質問セットを使用して潜在的なキーワードから選択肢を絞り込むことです。回答者は、Hugging FaceのLlama 8B LLMを使用して、与えられた質問に「はい」または「いいえ」で答えます。このシステムは、20の質問ゲームの形式を模倣し、特定のワードを特定するために効率的な質問を行うことが求められます。

### 主な手法と使用ライブラリ
- **Hugging Face Transformers:** LLM（Llama-3-Smaug-8B）を使用して質問に応答するために、`transformers`ライブラリから`AutoModelForCausalLM`および`AutoTokenizer`を使用しています。
- **PyTorch:** モデルの実行やテンソル処理のために`torch`ライブラリを使用します。
- **その他のライブラリ:** データ処理のために`pandas`、配列操作のために`numpy`を利用しています。また、モデルの量子化に`BitsAndBytesConfig`を活用し、メモリ効率を向上させています。

### フロー
1. **準備:** 必要なライブラリをインストールし、モデルを4bitの形式でダウンロードして保存します。
2. **キーワードの確認:** 指定されたキーワードがリストに存在するか確認し、カテゴリや大陸などの属性を基にプロンプトを生成します。
3. **質問応答:** モデルを使って、生成したプロンプトに対する応答を得ます。応答は「yes」または「no」にフォーマットされます。
4. **エージェント関数の実装:** `agent_fn(obs, cfg)` という関数を通じて、エージェントの動作を定義します。この関数では、質問者と回答者としてのロジックを組み込み、リアルタイムで情報を更新します。

### 目的と意義
このノートブックは、Kaggleの20の質問コンペティションへの参加に向けた基盤を提供するものであり、言語モデルとヒューリスティック手法を組み合わせることで、効率的にターゲットワードを推測するための戦略を示しています。データには、エージェントが質問に最適に応えるための知識ベースが含まれており、実践的な機械学習アプリケーションの意義を持つものです。

---


# 用語概説 
以下は、ノートブックに関連する機械学習・深層学習の専門用語であり、初心者がつまずく可能性があるものに対する簡単な解説です。特にマイナーなものや、実務を経験していないと馴染みのないものに焦点を当てています。

1. **Hugging Face**:
   Hugging Faceは、自然言語処理（NLP）や機械学習のためのオープンソースライブラリを提供する企業です。特に、Transformersライブラリが有名で、さまざまな事前学習済みモデルへのアクセスを可能にします。

2. **LLM (Large Language Model)**:
   大規模な言語モデルのことで、大量のデータを学習して言語を理解・生成する能力を持つモデルです。このノートブックでは、特にHugging FaceのLlamaモデルが使用されています。

3. **量子化 (Quantization)**:
   モデルのパラメータを低いビット数（例: 4bit）で保存し、計算の効率を上げる技術です。モデルのサイズを小さくしてメモリ使用量を削減しつつ、高速な推論を実現します。

4. **4bit量子化 (4-bit Quantization)**:
   通常の32ビットや16ビット浮動小数点数で表現されるパラメータを4ビットで表現する手法。これにより、モデルのサイズを大幅に削減し、計算資源を効率化します。ただし、精度は低下する可能性があります。

5. **Pad Token (パディングトークン)**:
   シーケンスデータの長さを揃えるために使用される特別なトークンで、短いシーケンスに無意味なデータを追加することで、同じサイズのバッチを作成するのに役立ちます。

6. **プロンプト (Prompt)**:
   モデルに対する入力文や指示文のことです。モデルはこのプロンプトに基づいて応答を生成します。質問やタスクを説明するために使われます。

7. **エージェント (Agent)**:
   あるタスクを実行するために設計されたプログラムやモデルのことを指します。ここでは、「質問者」と「回答者」として機能するLLMがエージェントとして扱われます。

8. **ヒューリスティック (Heuristic)**:
   問題解決のための経験則や簡単な手法のことです。特に、最適解を求めるのが難しい場合に、合理的な推論をするための方法として使われます。

9. **スキル評価 (Skill Rating)**:
   ゲームやコンペティションなどで、エージェントやプレイヤーの能力を数値化したものです。このコンペでは、エージェントのパフォーマンスを評価するために使用されます。

10. **CUDA (Compute Unified Device Architecture)**:
    NVIDIAが開発した並列計算プラットフォームおよびAPIで、GPUを用いた高速な計算を可能にします。PyTorchやTensorFlowなどの深層学習フレームワークで広く使われています。

これらの用語は、ノートブックでの実装やモデルの使用に関連しており、初心者が理解するのに役立つ情報です。

---


# Hugging Face Llama 8B LLM のためのスタートコード
これは、Kaggleの「20の質問」コンペティション用のHugging Face 8Bまたはそれ以下の大きさのLLMのためのスタートコードです。我々のエージェントは、`質問者`および`回答者`の両方として機能する必要があります。質問者に関しては、追加した新しい特徴を使って、潜在的なキーワードリストからバイナリ検索を行います（更新前）。**このノートブックは、キーワードが場所のみであったときに作成されました。**

このノートブックを実行する前に、更新前の全ての公的リーダーボード（LB）単語を含むCSVファイルを作成し、**場所**キーワードを検索するために使用できるいくつかの追加列を追加しました。追加した列は`大陸`と`最初の文字`です。回答者としてHugging FaceのLlama 8B LLMを使用します。

質問者の「質問」役割は**必ずしも**LLMを使用する必要はなく、むしろ検索を絞り込むための事前に決定された質問セットを使用できます。また、質問者の「推測」役割もヒューリスティックな論理を使用することができ、LLMである必要はありません。回答者は**必ず**LLMである必要があります。なぜなら、他のKaggleチームからの未知の質問に答える必要があるからです。

# 必要なライブラリのインストール
必要なパッケージはすべて、パス`/tmp/submission/lib`にインストールし、これを提出物のファイルとしてtarで圧縮します。

In [ ]:
!mkdir /tmp/submission  # /tmp/submissionディレクトリを作成します。このディレクトリは、後で提出物を格納するために使用されます。

In [ ]:
%%time  # このセルの実行時間を計測します。
import os, sys  # osモジュールとsysモジュールをインポートします。

# accelerateパッケージを/tmp/submission/libにインストールします。-Uオプションはアップグレードを意味します。
os.system("pip install -U -t /tmp/submission/lib accelerate")  
# bitsandbytesパッケージを指定したPyPIインデックスから/tmp/submission/libにインストールします。
os.system("pip install -i https://pypi.org/simple/ -U -t /tmp/submission/lib bitsandbytes")  
# pipのキャッシュをクリアします。これにより、次回以降のインストール時に古いバージョンが使用されないようにします。
os.system("pip cache purge")  
# カスタムライブラリのパスをPythonの実行環境に追加します。
sys.path.insert(0, "/tmp/submission/lib")

# LLMモデルのダウンロードと保存
モデルは`/tmp/submission/weights`にダウンロードし、保存します。ディスクとメモリの使用量を最小限に抑えるために、モデルは4bitで保存します。このモデルは、提出物のファイルにtarで圧縮されます。

このノートブックでは「Llama-3-Smaug-8B」を使用します。これは、追加データでファインチューニングされたLlama-3-8Bモデルです。詳細については、HuggingFaceの[こちら][1]を参照してください。

[1]: https://huggingface.co/abacusai/Llama-3-Smaug-8B

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig  # Hugging Faceのtransformersライブラリから必要なクラスをインポートします。
import torch  # PyTorchをインポートします。モデルの実行やテンソル操作に使用します。

In [ ]:
model_name = "abacusai/Llama-3-Smaug-8B"  # 使用するモデルの名前を定義します。

# 4bitでモデルを読み込むための設定を定義します。
bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,  # 4bitで読み込む設定
    bnb_4bit_quanty_type = "fp4",  # 量子化のタイプを指定
    bnb_4bit_compute_dtype = torch.float16,  # 演算に使用するデータ型を指定
    bnb_4bit_use_double_quanty = True,  # ダブル量子化を使用するかどうか
)

# 指定したモデル名で事前学習済みの言語モデルを読み込みます。
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config = bnb_config,  # 量子化設定を適用
    torch_dtype = torch.float16,  # モデルのデータ型を指定
    device_map = "auto",  # 自動でデバイスを設定
    trust_remote_code = True,  # リモートコードを信頼する設定
)

# モデルに関連付けられたトークナイザーを読み込みます。
tokenizer = AutoTokenizer.from_pretrained(model_name)  # トークナイザーも同じモデル名から取得します。

In [ ]:
model.save_pretrained("/tmp/submission/weights")  # 学習済みモデルを指定したパスに保存します。
tokenizer.save_pretrained("/tmp/submission/weights")  # トークナイザーを同じパスに保存します。これにより、モデルとトークナイザーの両方が後で利用可能になります。

# EDA LLMモデル
キーワードに関する質問に対して、我々のLLMがどのように答えるかを見てみましょう。このセクションでは、`回答者`に表示するカスタマイズされたプロンプトを決定するために使用します。

In [ ]:
import pandas as pd  # データ操作のためにpandasライブラリをインポートします。
# 更新されたKaggleキーワードのCSVファイルを読み込みます。
keywords = pd.read_csv("/kaggle/input/updated-kaggle-keywords/keywords_v2.csv")  
# 読み込んだデータの最初の行を表示します。
keywords.head(1)  # キーワードデータフレームの最初の1行を表示して、内容を確認します。

In [ ]:
pad_token_id = tokenizer.pad_token_id  # トークナイザーからパディングトークンのIDを取得します。
# パディングトークンがNoneの場合、終了トークンのIDを使用します。
if pad_token_id is None:
    pad_token_id = tokenizer.eos_token_id  # 終了トークンのIDをパディングトークンとして設定します。

In [ ]:
question = f"Is the secret word a country?"  # 質問を定義します。

keyword = "venezuela"  # キーワードを「venezuela」に設定します。
# キーワードがデータフレームの中に存在するか確認します。
if keyword in keywords.keyword.values:
    row = keywords.loc[keywords.keyword==keyword].iloc[0]  # キーワードに一致する行を取得します。
    category = row.category  # カテゴリーを取得します。例: "landmark"
    continent = row.continent  # 大陸を取得します。例: "North America"
    # 否定的な表現を定義します。
    negate = {
        "city": "Is it not a country. It is not a landmark.",
        "country": "Is it not a city. It is not a landmark.",
        "landmark": "Is it not a city. It is not a country.",
    }
    # プロンプトを作成します。
    prompt = f"We are playing 20 questions. The keyword is {keyword}. It is a {category}. {negate[category]} This word has first letter {keyword[0]}. This {category} is located in {continent}. {question}"
else:
    # キーワードが見つからない場合のプロンプトを設定します。
    prompt = f"We are playing 20 questions. The keyword is {keyword}. It is a thing. Is it not a city. It is not a country. It is not a landmark. This word has first letter {keyword[0]}. {question}"

# モデルに与えるメッセージを設定します。
messages = [
    {"role": "system", "content": "Answer yes or no to the following question and nothing else."},  # システムからの指示
    {"role": "user", "content": prompt}  # ユーザーからのプロンプト
]

# メッセージをトークナイザーに適用して、入力を準備します。
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to('cuda')  # 入力データをPyTorchテンソルに変換し、GPUに移動します。

# モデルを使用して応答を生成します。
generated_ids = model.generate(
    model_inputs.input_ids,
    attention_mask=model_inputs.attention_mask,
    pad_token_id=pad_token_id,
    max_new_tokens=1  # 最大生成トークン数を1に設定
)
# 生成されたIDから入力部を切り取ります。
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]  # 生成されたトークンIDをデコードします。
# 応答が「yes」でない場合は「no」とするロジック
if not "yes" in response.lower(): 
    response = "no"
else: 
    response = "yes"

# 結果を表示します。
print(f"When keyword is '{keyword}'")
print(f"and questioner asks: '{question}'")
print(f"with prompt = {prompt}")
print(f"our model answers: '{response}'")

In [ ]:
import gc, torch  # ガーベジコレクタとPyTorchをインポートします。

# 使用が終わったモデル、トークナイザー、モデル入力、生成されたID、応答を削除します。
del model, tokenizer, model_inputs, generated_ids, response  
# ガーベジコレクタを呼び出してメモリを解放します。
gc.collect()  
# CUDAのメモリキャッシュを空にします。
torch.cuda.empty_cache()

# エージェントの作成
私たちのエージェントファイルを`main.py`として保存します。エージェントは、`def agent_fn(obs, cfg)`という関数を実装する必要があります。

In [ ]:
!cp /kaggle/input/updated-kaggle-keywords/keywords_v2.csv /kaggle/working  # 更新されたKaggleキーワードのCSVファイルを現在の作業ディレクトリにコピーします。
!cp /kaggle/input/updated-kaggle-keywords/keywords_v2.csv /tmp/submission  # 同じCSVファイルを提出物のディレクトリにもコピーします。

In [ ]:
%%writefile /tmp/submission/main.py

################
# 送信またはコミットを決定する
import os
KAGGLE_AGENT_PATH = "/kaggle_simulations/agent/"
VERBOSE = False
if not os.path.exists(KAGGLE_AGENT_PATH + "weights"):
    KAGGLE_AGENT_PATH = "/tmp/submission/"
    VERBOSE = True

#################
# モデルをメモリに読み込む
import sys, torch
from transformers import AutoTokenizer, AutoModelForCausalLM
sys.path.insert(0, f"{KAGGLE_AGENT_PATH}lib")
model = AutoModelForCausalLM.from_pretrained(
    f"{KAGGLE_AGENT_PATH}weights/",
    torch_dtype = torch.float16,
    device_map = "auto",
    trust_remote_code = True,
)
tokenizer = AutoTokenizer.from_pretrained(f"{KAGGLE_AGENT_PATH}weights/")

##############
# 質問者としてのバイナリ検索
import pandas as pd, numpy as np
keywords = pd.read_csv(KAGGLE_AGENT_PATH + "keywords_v2.csv")
keywords['guess'] = 0

categories = ["city","country","landmark"]
#np.random.shuffle(categories)
category_yes = []
category_no = []
cat_guess = 0

continents = ["Europe","Asia","North America","Africa","South America","Australia"]
#np.random.shuffle(continents)
continent_yes = []
continent_no = []
con_guess = 0

first_letters = []
first_letter_yes = []
first_letter_no = []
let_guess = 0
extra_guess = ""

###############
# LLMモデルとしての回答者
def get_yes_no(question, keyword):
    global keywords, VERBOSE
    
    if keyword in keywords.keyword.values:
        row = keywords.loc[keywords.keyword==keyword].iloc[0]
        category = row.category #"landmark"
        continent = row.continent #"North America"
        negate = {
            "city":"これは国ではありません。それはランドマークではありません。",
            "country":"これは都市ではありません。それはランドマークではありません。",
            "landmark":"これは都市ではありません。それは国ではありません。",
        }
        prompt = f"We are playing 20 questions. The keyword is {keyword}. It is a {category}. {negate[category]} This word has first letter {keyword[0]}. This {category} is located in {continent}. {question}"
    else:
        prompt = f"We are playing 20 questions. The keyword is {keyword}. It is a thing. これは都市ではありません。それは国ではありません。それはランドマークではありません。この単語は最初の文字が{keyword[0]}です。{question}"
    
    messages = [
        {"role": "system", "content": "以下の質問に対して「はい」または「いいえ」で答えてください。"},
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to('cuda')
    
    pad_token_id = tokenizer.pad_token_id
    if pad_token_id is None:
        pad_token_id = tokenizer.eos_token_id
        
    generated_ids = model.generate(
        model_inputs.input_ids,
        attention_mask=model_inputs.attention_mask,
        pad_token_id=pad_token_id,
        max_new_tokens=1
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    if not "yes" in response.lower(): 
        response = "no"
    else: 
        response = "yes"
        
    if VERBOSE:
        print(f"### {prompt}")
        
    return response

############
# メインエージェント関数
def agent_fn(obs, cfg):
    global keywords, extra_guess, VERBOSE
    global categories, category_yes, category_no, cat_guess
    global continents, continent_yes, continent_no, con_guess
    global first_letters, first_letter_yes, first_letter_no, let_guess
        
    # 反応を生成する
    if obs.turnType == "ask":
        
        if (cat_guess < 3) & (len(category_yes) == 0):
            response = f"キーワードは{categories[cat_guess]}の名前ですか？"
            cat_guess += 1
        elif (con_guess < 6) & (len(continent_yes) == 0):
            category = "場所"
            if len(category_yes) == 1: 
                category = category_yes[0]
            response = f"{category}は{continents[con_guess]}に位置していますか？"
            con_guess += 1
        else:
            IDX = keywords.category.isin(category_yes)
            IDX = IDX & (keywords.continent.isin(continent_yes))
            first_letters = list(keywords.loc[IDX, "first_letter"].value_counts().index.values)
            if let_guess < len(first_letters):
                response = f"キーワードの最初の文字は{first_letters[let_guess]}ですか？"
            else:
                IDX = keywords.guess == 0
                if len(category_yes) > 0: IDX = IDX & (keywords.category.isin(category_yes))
                if len(category_no) > 0: IDX = IDX & (~keywords.category.isin(category_no))
                if len(continent_yes) > 0: IDX = IDX & (keywords.continent.isin(continent_yes))
                if len(continent_no) > 0: IDX = IDX & (~keywords.continent.isin(continent_no))
                if len(first_letter_yes) > 0: 
                    IDX = IDX & (keywords.first_letter.isin(first_letter_yes))
                if len(first_letter_no) > 0: 
                    IDX = IDX & (~keywords.first_letter.isin(first_letter_no))
                try:
                    guess = keywords.loc[IDX].sample(1).index.values[0]
                    keywords.loc[guess, 'guess'] = 1
                    response = keywords.loc[guess, "keyword"]
                except:
                    response = np.random.choice(keywords.keyword.values)
                extra_guess = response
                response = f"それは{response}ですか？"
            let_guess += 1
            
    elif obs.turnType == "guess":
        
        category_yes = []
        category_no = []
        for k in range(cat_guess):
            if obs.answers[k] == "yes":
                category_yes.append(categories[k])
            else:
                category_no.append(categories[k])
        if (cat_guess == 3) & (len(category_yes) == 0):
            category_yes = ["city", "country", "landmark"]
            category_no = []
            
        continent_yes = []
        continent_no = []
        for k in range(con_guess):
            if obs.answers[k + cat_guess] == "yes":
                continent_yes.append(continents[k])
            else:
                continent_no.append(continents[k])
        if (con_guess == 6) & (len(continent_yes) == 0):
            continent_yes = ["Europe", "Asia", "North America", "Africa", "South America", "Australia"]
            continent_no = []
            
        first_letter_yes = []
        first_letter_no = []
        for k in range(let_guess):
            if k >= len(first_letters): continue
            if obs.answers[k + cat_guess + con_guess] == "yes":
                first_letter_yes.append(first_letters[k])    
            else:
                first_letter_no.append(first_letters[k]) 
                
        IDX = keywords.guess == 0
        if len(category_yes) > 0: IDX = IDX & (keywords.category.isin(category_yes))
        if len(category_no) > 0: IDX = IDX & (~keywords.category.isin(category_no))
        if len(continent_yes) > 0: IDX = IDX & (keywords.continent.isin(continent_yes))
        if len(continent_no) > 0: IDX = IDX & (~keywords.continent.isin(continent_no))
        if len(first_letter_yes) > 0: IDX = IDX & (keywords.first_letter.isin(first_letter_yes))
        if len(first_letter_no) > 0: IDX = IDX & (~keywords.first_letter.isin(first_letter_no))
            
        try:
            guess = keywords.loc[IDX].sample(1).index.values[0]
            keywords.loc[guess, 'guess'] = 1
            response = keywords.loc[guess, "keyword"]
        except:
            response = np.random.choice(keywords.keyword.values)
            
        if (let_guess > 0) & (let_guess >= len(first_letters)) & (obs.answers[-1] == "yes"):
            response = extra_guess
        
    else:  # obs.turnType == "answer"
        if obs.keyword.lower() in obs.questions[-1].lower():
            response = "yes"
        else:
            response = get_yes_no(obs.questions[-1], obs.keyword)
            
    # 役割を表示
    if VERBOSE: 
        if obs.turnType == "answer": 
            print(f"チーム2 - 回答者 - ### エージェント LLAMA 8B ###")
        else:
            print(f"\nチーム2 - 質問者 - ### エージェント LLAMA 8B ###")
        print(f"OUTPUT = '{response}'")

    return response

# 提出物のタールボールを保存
これで、ライブラリ、モデル、およびmain.pyをタールボールに保存します。

In [ ]:
!apt install pigz pv > /dev/null  # pigzおよびpvをインストールします。これにより、圧縮処理が高速化され、進捗状況が表示されます。

In [ ]:
!tar --use-compress-program='pigz --fast --recursive | pv' -cf /kaggle/working/submission.tar.gz -C /tmp/submission .  # /tmp/submission内の全てのファイルを圧縮し、進捗状況を表示しながら/kaggle/workingにsubmission.tar.gzとして保存します。

# KaggleのAPIに関するEDA
以下に、KaggleのAPIに関するEDAを示します。ここでは、ランダムに選んだ3つのエージェントと我々のLlama 8B LLMエージェントを一緒に実行します。

Kaggleの「20の質問」コンペティションでは、各チャレンジは4つのKaggleチームで構成されています。これは2つのチーム対2つのチームの対戦です。このグループのチームをGroupTeam A対GroupTeam Bと呼びましょう。

一つのGroupTeam内では、「質問者」（4つのKaggleチームのうちの1チーム）と「回答者」（4つのKaggleチームのうちの1チーム）があります。「質問者」の役割は2つ（すなわち、質問することと推測すること）であり、「回答者」の役割は1つ（すなわち、答えること）です。

各ラウンドで、「質問者」が質問を行い、その後グループの仲間である「回答者」が「はい」または「いいえ」で答えます。最後に、「質問者」が単語を推測します。これを20ラウンド繰り返します。

In [ ]:
import pandas as pd  # データ操作のためにpandasライブラリをインポートします。

# データフレームを表示
df = pd.read_csv("/kaggle/input/updated-kaggle-keywords/keywords_v2.csv")  # 更新されたKaggleキーワードのCSVファイルを読み込みます。
n = df.category.nunique()  # ユニークなカテゴリ数を取得します。
print(f"\n更新前の公的リーダーボードでは、{len(df)}件のキーワードと{n}カテゴリがあります。")  # キーワード数とカテゴリ数を表示します。
print("以下は20件のランダムな例です:\n")
df.sample(20, random_state=42)  # データフレームから20件のランダムサンプルを表示します。

In [ ]:
%%writefile /kaggle/working/agent1.py

import pandas as pd, numpy as np  # pandasとnumpyをインポートします。
# 事前に定義されたキーワードを読み込みます。
keywords = pd.read_csv("keywords_v2.csv").keyword.values

def agent_fn(obs, cfg):  # エージェントのメイン関数を定義します。
    global keywords  # グローバル変数としてkeywordsを指定します。
    
    # ラウンド番号を表示
    k = len(obs.questions)  # 現在の質問数を取得します。
    if obs.turnType == "ask":  # 質問のターンの場合
        print()
        print("#"*25)
        print(f"### Round {k+1}")  # 次のラウンド番号を表示
        print("#"*25)

    # エージェント名とJSON入力を表示
    name = "Team 1 - Questioner - Agent Random"  # エージェント名を設定
    print(f"\n{name}\nINPUT =", obs)  # 入力を表示
    
    # 応答を生成
    keyword = np.random.choice(keywords)  # ランダムにキーワードを選択します。
    if obs.turnType == "ask":  # 質問のターンの場合
        response = f"Is it {keyword}?"  # 質問として生成します。
    else:  # obs.turnType == "guess"の場合
        response = keyword  # 選ばれたキーワードを応答として設定します。
        if obs.answers[-1] == "yes":  # 最後の応答が「はい」の場合
            response = obs.questions[-1].rsplit(" ", 1)[1][:-1]  # 最後の質問からキーワードを抽出します。
    print(f"OUTPUT = '{response}'")  # 出力を表示

    return response  # 応答を返します。

In [ ]:
%%writefile /kaggle/working/agent2.py

import numpy as np  # numpyライブラリをインポートします。

def agent_fn(obs, cfg):  # エージェントのメイン関数を定義します。
    
    # エージェント名とJSON入力を表示
    name = "Team 1 - Answerer - Agent Random"  # エージェント名を設定
    print(f"\n{name}\nINPUT =", obs)  # 入力を表示
    
    # 応答を生成
    response = "no"  # デフォルト応答を「いいえ」に設定
    #response = np.random.choice(["yes","no"])  # ランダムに「はい」または「いいえ」を選択することも可能
    if obs.keyword.lower() in obs.questions[-1].lower():  # キーワードが最後の質問に含まれている場合
        response = "yes"  # 応答を「はい」に変更
    print(f"OUTPUT = '{response}'")  # 出力を表示

    return response  # 応答を返します。

In [ ]:
%%writefile /kaggle/working/agent3.py

import pandas as pd, numpy as np  # pandasとnumpyをインポートします。
# 事前に定義されたキーワードを読み込みます。
keywords = pd.read_csv("keywords_v2.csv").keyword.values

def agent_fn(obs, cfg):  # エージェントのメイン関数を定義します。
    global keywords  # グローバル変数としてkeywordsを指定します。
    
    # エージェント名とJSON入力を表示
    name = "Team 2 - Questioner - Agent Random"  # エージェント名を設定
    print(f"\n{name}\nINPUT =", obs)  # 入力を表示
    
    # 応答を生成
    keyword = np.random.choice(keywords)  # ランダムにキーワードを選択します。
    if obs.turnType == "ask":  # 質問のターンの場合
        response = f"Is it {keyword}?"  # 質問として生成します。
    else:  # obs.turnType == "guess"の場合
        response = keyword  # 選ばれたキーワードを応答として設定します。
        if obs.answers[-1] == "yes":  # 最後の応答が「はい」の場合
            response = obs.questions[-1].rsplit(" ", 1)[1][:-1]  # 最後の質問からキーワードを抽出します。
    print(f"OUTPUT = '{response}'")  # 出力を表示

    return response  # 応答を返します。

In [ ]:
%%writefile /kaggle/working/agent4.py

import numpy as np  # numpyライブラリをインポートします。

def agent_fn(obs, cfg):  # エージェントのメイン関数を定義します。
    
    # エージェント名とJSON入力を表示
    name = "Team 2 - Answerer - Agent Random"  # エージェント名を設定
    print(f"\n{name}\nINPUT =", obs)  # 入力を表示
    
    # 応答を生成
    response = "no"  # デフォルトの応答として「いいえ」を設定
    #response = np.random.choice(["yes","no"])  # ランダムに「はい」または「いいえ」を選択するためのオプション
    if obs.keyword.lower() in obs.questions[-1].lower():  # キーワードが最後の質問に含まれている場合
        response = "yes"  # 応答を「はい」に変更
    print(f"OUTPUT = '{response}'")  # 出力を表示

    return response  # 応答を返します。

In [ ]:
!pip install -q pygame  # pygameライブラリを静かにインストールします。これはゲームやマルチメディアアプリケーションの開発に使用されるライブラリです。

In [ ]:
LLAMA_AS_QUESTIONER = True  # LLMを質問者として使用するかどうかのフラグ
LLAMA_AS_ANSWERER = True  # LLMを回答者として使用するかどうかのフラグ

from kaggle_environments import make  # Kaggle環境を作成するためにインポートします。
env = make("llm_20_questions", debug=True)  # 20 QuestionsのKaggle環境を作成し、デバッグモードで動作させます。

# チーム1のエージェント
agent1 = "/kaggle/working/agent1.py"  # チーム1の質問者エージェント
agent2 = "/kaggle/working/agent2.py"  # チーム1の回答者エージェント

# チーム2 - 質問者
agent3 = "/kaggle/working/agent3.py"  # チーム2の質問者エージェント
if LLAMA_AS_QUESTIONER:
    agent3 = "/tmp/submission/main.py"  # LLMを質問者として使用する場合

# チーム2 - 回答者
agent4 = "/kaggle/working/agent4.py"  # チーム2の回答者エージェント
if LLAMA_AS_ANSWERER:
    agent4 = "/tmp/submission/main.py"  # LLMを回答者として使用する場合
    
env.reset()  # 環境をリセット
log = env.run([agent1, agent2, agent3, agent4])  # エージェントを実行してログを取得

import gc, torch  # ガーベジコレクタとPyTorchをインポート
# 使用が終わったモジュールやオブジェクトを削除
del make, env, log  
gc.collect()  # ガーベジコレクタを呼び出してメモリを解放
torch.cuda.empty_cache()  # CUDAのメモリキャッシュを空にします。

In [ ]:
# 提出しないためにシミュレーションエージェントを削除します
!rm agent1.py agent2.py agent3.py agent4.py keywords_v2.csv  # エージェントファイルとキーワードファイルを削除します。

# コメント 

> ## shiv_314
> 
> このノートブックは素晴らしいです！8Bパラメータを上限として使用できるという事実をどうやって思いついたのですか？その推計のプロセスを説明していただけますか？
> 
> > ## Chris DeotteTopic Author
> > 
> > 1xT4 GPUを使用することで、実際には17Bモデルを推論できるのですが（34Bモデルを2xT4で推論する方法を[こちら](https://www.kaggle.com/code/cdeotte/infer-34b-with-vllm)で示しています）、17Bモデルから始めると4bit量子化後にディスク上で10GBになり、1xT4の16GB VRAMに容易に収まります。（更新：もしかしたら、20Bモデルを推論できるかもしれません）
> > 
> 

---

> ## Istvan Habram
> 
> エージェントがdef agent_fn(obs, cfg)という関数を実装する必要がありますというのはどこに指定されていますか？
> 
> > ## Chris DeotteTopic Author
> > 
> > これはKaggle管理者のスターターノートブック[こちら](https://www.kaggle.com/code/ryanholbrook/llm-20-questions-starter-notebook)や、KaggleのGitHub[こちら](https://github.com/Kaggle/kaggle-environments/tree/master/kaggle_environments/envs/llm_20_questions)を読むことによって学びます。そこで私たちのエージェントを実行するコードが示されています。
> > 
> > 
> > > ## Istvan Habram
> > > 
> > > なるほど、今はっきりしました。迅速な返答ありがとうございます！
> > > 
> > > 

---

> ## Toshi
> 
> 素晴らしいノートブックですね！
> 
> これを実行したとき、以下のエラーが見つかりました。
> 
> ImportError: numpy>=1.17,<2.0 is required for a normal functioning of this module, but found numpy==2.0.0.
> 
> Try: pip install transformers -U or pip install -e '.[dev]' if you're working with git main
> 
> これを実行したときに同じエラーを見つけましたか？
> 
> > ## Chris DeotteTopic Author
> > 
> > いいえ。そのエラーを見たことはありません。このノートブックをそのまま実行していますか？それとも何かを変更していますか？ノートブックオプションの環境を「元に固定」に設定していますか？それともKaggleの最新のdockerを使用していますか？Kaggleの最新がエラーを引き起こしている場合は「元に固定」を使用するべきです。
> > 
> > 何かを変更してnumpy==2.0.0を使用できない原因になっている場合、例えばpip install numpy==1.17のように古いnumpyをインストールしてみることができます。
> > 
> > 
> > > ## FullEmpty
> > > 
> > > [@cdeotte](https://www.kaggle.com/cdeotte) ノートブックをありがとう - あなたの投稿やコメントからいつも多くを学びます。Kaggleノートブックの習得には至っていませんが、このノートブックはコピーして実行すると動作します。しかし、ダウンロードしてインポートして実行するとき、元に固定環境でも上述のエラーが発生します。
> > > 
> > > しかし、問題は以下のように解決できます：
> > > 
> > > !rm -rf /tmp/submission/lib/numpy-2.0.1.dist-info
> > > 
> > > または、好ましくは、
> > > 
> > > os.system("pip install accelerate==0.33.0 bitsandbytes==0.43.2 huggingface_hub==0.24.2 -t /tmp/submission/lib")
> > > 
> > > 

---

> ## Kartikeya Pandey
> 
> 素晴らしいノートブックです。多くを学びました。
> 
> 

---

> ## El haddad Mohamed
> 
> 素晴らしいノートブック！非常に構造がしっかりしており、情報が豊富です。👍
> 
> 

---

> ## ISAKA Tsuyoshi
> 
> こんにちは[@cdeotte](https://www.kaggle.com/cdeotte)。この優れたノートブックを共有していただきありがとうございます！
> 
> このノートブックで使用されているモデル（Llama-3-Smaug-8B）についてもっと教えていただけますか？これは llama-3 モデルより優れていますか？
> 
> > ## Chris DeotteTopic Author
> > 
> > Llama-3-Smaug-8Bは、追加のファインチューニングが施されたLlama-3-8B-Instructモデルです。いくつかの点でLlama-3-8B-Instructより優れていますが、おそらくいくつかの点では劣っています。Smaugのトレーニング方法を説明している研究論文は[こちら](https://arxiv.org/abs/2402.13228)にあります。このモデルを発見したのは、ある時点でこのモデルが従来のLlamaよりもOpen LLM Leaderboardでスコアが高かったからです。
> > 
> > もう一つ、かつてOpen LLM Leaderboardで好成績を収めた興味深いモデルは jondurbin/bagel-7b-v0.5 [こちら](https://huggingface.co/jondurbin/bagel-7b-v0.5)です。これはMistral-7Bの追加ファインチューニング版です。Bagelのトレーニング方法を説明するGitHubは[こちら](https://github.com/jondurbin/bagel)にあります。
> > 
> > そして、Open LLM Leaderboardで好成績のもう一つのモデルはQwen/Qwen2-7B-Instruct [こちら](https://huggingface.co/Qwen/Qwen2-7B-Instruct)です。私はQwen2モデルの大ファンです。このv2シリーズが先月リリースされたばかりで（以前はv1.5シリーズでした）、すぐにOpen LLM Leaderboardモデルのトップになりました。
> > 
> > （完全性のために、古典的なモデルとしてはLlama3、Mistral、Gemma2、Phi3を挙げておきます）
> > 
> > 
> > > ## ISAKA Tsuyoshi
> > > 
> > > 詳細な説明をありがとうございます！選べるオプションがたくさんありますね。
> > > 
> > > 